# Classifier for Kaggle popcorn competition

Uses logistic regression to try and predict sentiment from a bag-of-words summary of each review.

In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
# Expit is just the sigmoid function, written in C
from scipy.special import expit

data = pd.read_csv('even_better_features.csv')

ys = data.iloc[:, 0]
data = data.iloc[:, 1:]

In [2]:
data['ones_column'] = np.ones_like(ys)

In [3]:
from functools import wraps
from timeit import default_timer

def timed(f):
    """Decorator to measure the runtime of a function and print it to
    stdout. May not be required."""
    if hasattr(f, 'func_name'):
        name = f.func_name
    elif hasattr(f, 'im_func'):
        name = f.im_func.func_name
    elif hasattr(f, '__name__'):
        name = f.__name__
    else:
        name = '<unknown>'

    @wraps(f)
    def inner(*args, **kwargs):
        print("Calling {}".format(name))
        start = default_timer()
        rv = f(*args, **kwargs)
        taken = default_timer() - start
        print("Call to {} took {}s".format(name, taken))
        return rv

    return inner

In [4]:
def safe_sigmoid(x):
    """Computes sigmoid function, but in the numerically stable way used by Sklearn"""
    gt_zero = x > 0
    gt_zero_x = x[gt_zero]
    not_gt_zero_x = x[~gt_zero]
    rv = np.zeros_like(x)
    rv[gt_zero] = 1.0 / (1 + np.exp(-gt_zero_x))
    rv[~gt_zero] = np.exp(not_gt_zero_x) / (np.exp(not_gt_zero_x) + 1)
    return rv

def safe_log_sigmoid(x):
    """Compute log(sigmoid(x)) safely"""
    rv = np.zeros(len(x))
    mask = x > 0
    rv[mask] = -np.log(1 + np.exp(-x[mask]))
    rv[~mask] = x[~mask] - np.log(1 + np.exp(x[~mask]))
    return rv

def safe_log_one_minus_sigmoid(x):
    """Compute log(1 - sigmoid(x)) safely"""
    rv = np.zeros(len(x))
    mask = x > 0
    rv[mask] = -x[mask] - np.log(1 + np.exp(-x[mask]))
    rv[~mask] = -np.log(np.exp(x[~mask]) + 1)
    return rv

def logistic_gradient(weights, features, labels):
    """Compute the gradient of the cross-entropy error for logistic regression"""
    predictions = expit(np.dot(features, weights))
    error = predictions - labels
    # We divide by label.size to try and keep numerical error under control
    rv = np.dot(error, features) / labels.size
    return rv

def cross_entropy_error(weights, features, labels):
    """Cross-entropy error for logistic regression"""
    dots = np.dot(features, weights)
    vect = labels*safe_log_sigmoid(dots) + (1-labels)*safe_log_one_minus_sigmoid(dots)
    rv = -np.mean(vect)
    return rv

def logistic_predict(weights, features):
    """Predict labels for the given features using the given weights"""
    probs = expit(np.dot(features, weights))
    rv = np.zeros(len(features))
    rv[probs > 0.5] = 1
    return rv

def train_logistic_regression(labels, features):
    """Find a weight vector for logistic regression using the supplied
    labels and features."""
    np_labels = np.array(labels)
    
    initial_w = 0.1 * np.random.random(features.shape[1])
    assert initial_w.ndim == 1
    
    result = opt.fmin_cg(
        cross_entropy_error, 
        initial_w, 
        fprime=logistic_gradient,
        args=(features, np_labels),
        maxiter=20
    )
    
    return result

In [5]:
print(cross_entropy_error(np.random.random(data.shape[1]), data, ys))
print(logistic_gradient(np.random.random(data.shape[1]), data, ys))

21.2889745619
[ 0.0038      0.0012      0.0026     ...,  0.0186      0.0026      0.49656414]


In [6]:
# ws = train_logistic_regression(ys, data)
# predictions = logistic_predict(ws, data)
# MAXIMUM OVERFIT
# np.sum(predictions != ys)

# Classifier using scikit-learn

The following classifier uses scikit-learn's SVM implementation to predict whether the movie gets a thumbs up or thumbs down.

In [7]:
from sklearn import svm

# From example
# clf = svm.SVC()
# clf.fit(data[:2500], ys[:2500])
# predictions = clf.predict(data[2500:])
# print("Correct/total: {}/{}".format(
#     np.sum(predictions == ys[2500:]), len(ys[:2500])
# ))

# AdaBoost!

In [11]:
class AdaBoostLearner(object):
    def fit(self, weights, labels, features):
        pass
    
    def predict(self, features):
        pass
    
class DecisionStump(AdaBoostLearner):
    def __init__(self, axis=None):
        self.axis = axis
        # Is the class 1 mean greater than the class 0 mean?
        self.greater = False
        self.threshold = None

    def fit(self, weights, labels, features):
        if self.axis is None:
            self.axis = np.random.randint(len(features))
        ones_mask = labels == 1
        ones_feats = features[ones_mask,self.axis]
        ones_mean = np.mean(ones_feats)
        zeros_feats = features[~ones_mask,self.axis]
        zeros_mean = np.mean(zeros_feats)
        self.greater = ones_mean >= zeros_mean
        self.threshold = np.mean([ones_mean, zeros_mean])
    
    def predict(self, features):
        assert self.axis is not None, "Train to get axis"
        axis_values = features[:,self.axis].flatten()
        rv = np.zeros(len(features))
        mask = axis_values >= self.threshold if self.greater else axis_values < self.threshold
        rv[mask] = 1
        return rv
    
class SVMLearner(AdaBoostLearner):
    def fit(self, weights, labels, features):
        self.clf = svm.SVC()
        normed = weights / np.sum(weights)
        indices = np.random.choice(
            np.arange(len(weights)),
            size=len(weights),
            replace=True,
            p=normed
        )
        self.clf.fit(features[indices], labels[indices])
    
    def predict(self, features):
        rv = self.clf.predict(features)
        return rv

def train_adaboost(labels, features, learners=tuple()):
    weights = np.ones_like(labels) / len(features)
    alphas = np.zeros(len(learners))
    for m in range(len(learners)):
        learners[m].fit(weights, labels, features)
        preds = learners[m].predict(features)
        mp = preds != labels
        epsilon = np.sum(weights[mp]) / np.sum(weights)
        if epsilon == 0:
            # OMG WE HAVE A PERFECT CLASSIFIER
            break
        alphas[m] = np.log((1 - epsilon) / epsilon)
        weights[mp] *= np.exp(alphas[m])
    return alphas

def adaboost_predict(alphas, learners, features):
    preds = np.zeros((len(alphas), len(features)))
    for m in range(len(learners)):
        preds[m] = learners[m].predict(features)
    lc = np.dot(alphas, preds)
    rv = np.zeros(len(features))
    rv[lc > 0] = 1
    return rv

real_data = data.as_matrix()
real_ys = ys.as_matrix()
train_set = (real_ys[:2500], real_data[:2500])
test_set = (real_ys[2500:], real_data[2500:])
learners = [SVMLearner() for m in range(5)]
alphas = train_adaboost(train_set[0], train_set[1], learners)
preds = adaboost_predict(alphas, learners, test_set[1])
correct = np.sum(preds == test_set[0])
print("Correct: {}/{}".format(correct, len(preds)))

Correct: 1500/2500
